<a href="https://colab.research.google.com/github/Bernschu/quant-trading/blob/main/renko_trading_multimetrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Matplotlib-Konfiguration für CURSOR
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# Imports
from typing import Tuple, Dict, Optional, List, Any
import pandas as pd
import numpy as np
import yfinance as yf
import logging
from datetime import datetime, timedelta
from dataclasses import dataclass
import warnings
import tkinter as tk
from tkinter import messagebox
import time
import itertools
import seaborn as sns
import feedparser
import requests
import ta
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')

@dataclass
class RenkoParams:
    """Parameters for Renko chart calculation."""
    atr_period: int = 14
    brick_multiplier: float = 1.0
    chart_history: int = 500

class RenkoChart:
    def __init__(self, atr_period: int = 14):
        self.atr_period = atr_period
        self.bricks = []

    def calculate_atr(self, data: pd.DataFrame) -> float:
        """Calculate Average True Range."""
        high = data['High']
        low = data['Low']
        close = data['Close'].shift(1)

        tr1 = high - low
        tr2 = abs(high - close)
        tr3 = abs(low - close)

        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(window=self.atr_period).mean().iloc[-1]

        return atr

    def build_renko_chart(self, data: pd.DataFrame) -> List[Dict]:
        """Build Renko chart from price data."""
        self.brick_size = self.calculate_atr(data)
        close_prices = data['Close'].values

        if not self.bricks:
            # Initialize first brick
            self.bricks.append({
                'open': close_prices[0],
                'close': close_prices[0],
                'direction': 0,
                'timestamp': data.index[0]
            })

        for i, price in enumerate(close_prices):
            self.update_renko(price, data.index[i])

        return self.bricks

    def update_renko(self, price: float, timestamp: datetime) -> None:
        """Update Renko chart with new price."""
        last_brick = self.bricks[-1]

        while True:
            if price >= last_brick['close'] + self.brick_size:
                # New up brick
                new_brick = {
                    'open': last_brick['close'],
                    'close': last_brick['close'] + self.brick_size,
                    'direction': 1,
                    'timestamp': timestamp
                }
                self.bricks.append(new_brick)
                last_brick = new_brick
            elif price <= last_brick['close'] - self.brick_size:
                # New down brick
                new_brick = {
                    'open': last_brick['close'],
                    'close': last_brick['close'] - self.brick_size,
                    'direction': -1,
                    'timestamp': timestamp
                }
                self.bricks.append(new_brick)
                last_brick = new_brick
            else:
                break

class RenkoStrategy:
    """Erweiterte Renko-Handelsstrategie mit Performance-Optimierungen."""

    def __init__(self, symbol: str, atr_period: int = 14,
                 risk_per_trade: float = 0.02,
                 max_position_size: float = 1.0,
                 max_drawdown: float = 0.15,
                 max_portfolio_exposure: float = 0.75,
                 account_size: float = 100000.0):

        # Bestehende Initialisierung
        self.symbol = symbol
        self.renko = RenkoChart(atr_period=atr_period)
        self.risk_per_trade = risk_per_trade
        self.max_position_size = max_position_size
        self.max_drawdown = max_drawdown
        self.max_portfolio_exposure = max_portfolio_exposure
        self.account_size = account_size
        self.current_drawdown = 0.0
        self.current_exposure = 0.0
        self.positions = []
        self.performance_metrics = {}

        # Neue Optimierungs-Komponenten
        self.dynamic_brick_sizer = DynamicBrickSizer()
        self.enhanced_signal_generator = EnhancedSignalGenerator()
        self.adaptive_position_sizer = AdaptivePositionSizer()
        self.multi_timeframe_analyzer = MultiTimeframeAnalyzer()
        self.trend_analyzer = EnhancedTrendAnalyzer()
        self.exit_manager = AdaptiveExitManager()
        self.ml_enhancer = MLSignalEnhancer()
        self.strategy_optimizer = StrategyOptimizer()

        # Neue Prognose-Engine
        self.prediction_engine = PricePredictionEngine()

    def generate_trading_signal(self, data: pd.DataFrame) -> Dict[str, Any]:
        """Erweiterte Signalgenerierung mit allen Optimierungen."""
        try:
            # 1. Ursprüngliche Renko-Analyse (BEIBEHALTEN)
            renko_bricks = self.renko.build_renko_chart(data)
            base_signal = self.analyze_renko_pattern(renko_bricks)

            # 2. Multi-Timeframe-Analyse (NEU)
            mtf_signal = self.multi_timeframe_analyzer.analyze_timeframes(self.symbol)

            # 3. Trend-Analyse (NEU)
            trend_data = self.trend_analyzer.analyze_trend_strength(data)

            # 4. Erweiterte Signalgenerierung (NEU)
            enhanced_signal = self.enhanced_signal_generator.generate_enhanced_signal(
                renko_bricks, data)

            # 5. ML-Verbesserung (NEU)
            final_signal = self.ml_enhancer.enhance_signal(enhanced_signal, data)

            # Kombiniere alle Signale (NEU)
            combined_signal = self.combine_signals(
                base_signal=base_signal,
                mtf_signal=mtf_signal,
                trend_data=trend_data,
                enhanced_signal=enhanced_signal,
                final_signal=final_signal)

            return combined_signal

        except Exception as e:
            logger.error(f"Error generating trading signal: {str(e)}")
            return base_signal  # Fallback auf ursprüngliches Signal

    def execute_trade(self, signal: Dict[str, Any], data: pd.DataFrame) -> None:
        """Erweiterte Handelsausführung mit optimierten Komponenten."""
        try:
            # 1. Dynamische Positionsgrößenberechnung (NEU)
            position_size = self.adaptive_position_sizer.calculate_position_size(
                signal_strength=signal['strength'],
                volatility=self.calculate_volatility(data),
                trend_strength=signal['trend_strength']
            )

            # 2. Exit-Management Setup (NEU)
            exit_points = self.exit_manager.calculate_exit_points(
                entry_price=data['Close'].iloc[-1],
                trend_data=signal['trend_data'],
                volatility=self.calculate_volatility(data)
            )

            # 3. Ursprüngliche Handelsausführung (BEIBEHALTEN)
            if self.validate_trade(signal, position_size):
                self.place_trade(
                    signal=signal,
                    position_size=position_size,
                    exit_points=exit_points
                )

        except Exception as e:
            logger.error(f"Error executing trade: {str(e)}")

    def optimize_strategy(self) -> None:
        """Kontinuierliche Strategieoptimierung."""
        try:
            # 1. Performance-Analyse
            current_metrics = self.calculate_performance_metrics()

            # 2. Parameter-Optimierung
            if current_metrics['sharpe_ratio'] < self.target_metrics['sharpe_ratio']:
                self.strategy_optimizer.optimize_parameters()

            # 3. Signal-Schwellwerte anpassen
            if current_metrics['win_rate'] < self.target_metrics['win_rate']:
                self.strategy_optimizer.adjust_signal_thresholds()

            # 4. Position Sizing überprüfen
            if current_metrics['max_drawdown'] > self.target_metrics['max_drawdown']:
                self.strategy_optimizer.adjust_position_sizing()

        except Exception as e:
            logger.error(f"Error optimizing strategy: {str(e)}")

    def calculate_volatility_adjusted_position_size(self, price: float,
                                                  stop_loss: float,
                                                  volatility: float) -> float:
        """Calculate position size based on volatility."""
        try:
            risk_amount = self.account_size * self.risk_per_trade
            price_risk = abs(price - stop_loss)

            if price_risk == 0:
                return 0

            base_position_size = risk_amount / price_risk
            vol_factor = 1.0 / (volatility * np.sqrt(252)) if volatility > 0 else 1.0
            adjusted_size = base_position_size * vol_factor

            # Exposure-Prüfung
            remaining_exposure = self.max_portfolio_exposure - self.current_exposure
            max_allowed_size = remaining_exposure * self.account_size / price

            # Drawdown-Anpassung
            if self.current_drawdown > self.max_drawdown * 0.5:
                drawdown_factor = 1 - (self.current_drawdown / self.max_drawdown)
                adjusted_size *= drawdown_factor

            return min(adjusted_size, self.max_position_size, max_allowed_size)

        except Exception as e:
            logger.error(f"Error calculating position size: {str(e)}")
            return 0.0

    def generate_signals(self, renko_data: List[Dict]) -> List[Dict]:
        """Generate trading signals from Renko chart."""
        signals = []
        for i in range(3, len(renko_data)):
            bricks = renko_data[i-3:i+1]

            # Buy signal: Three consecutive up bricks
            if all(b['direction'] == 1 for b in bricks[:-1]):
                signals.append({
                    'timestamp': bricks[-1]['timestamp'],
                    'type': 'BUY',
                    'price': bricks[-1]['close'],
                    'confidence': self.calculate_signal_confidence(bricks, 'BUY')
                })

            # Sell signal: Three consecutive down bricks
            elif all(b['direction'] == -1 for b in bricks[:-1]):
                signals.append({
                    'timestamp': bricks[-1]['timestamp'],
                    'type': 'SELL',
                    'price': bricks[-1]['close'],
                    'confidence': self.calculate_signal_confidence(bricks, 'SELL')
                })

        return signals

    def calculate_signal_confidence(self, bricks: List[Dict], signal_type: str) -> float:
        """Calculate confidence level of trading signal."""
        # Momentum der Bricks
        momentum = sum(b['direction'] for b in bricks)
        # Größe der Bricks
        size_factor = sum(abs(b['close'] - b['open']) for b in bricks) / (len(bricks) * self.renko.brick_size)
        # Zeitliche Komponente
        time_factor = 1.0
        if len(bricks) > 1:
            duration = (bricks[-1]['timestamp'] - bricks[0]['timestamp']).total_seconds()
            time_factor = min(1.0, 86400 / duration) if duration > 0 else 0.5

        confidence = (0.4 * abs(momentum) / len(bricks) +
                     0.4 * size_factor +
                     0.2 * time_factor)

        return min(0.99, confidence)

    def get_current_prediction(self, model: Any, scaler: Any) -> Dict[str, Any]:
        """Generate trading recommendation."""
        try:
            latest_data = self.add_technical_indicators(self.stock_data).tail(1)

            if latest_data.empty:
                raise ValueError("No current data available")

            current_features = latest_data[self.features]
            X_scaled = scaler.transform(current_features)

            prediction = model.predict(X_scaled)[0]
            probability = model.predict_proba(X_scaled)[0]
            confidence = float(max(probability))
            current_price = self.stock_data['Close'].iloc[-1]

            recommendation = {
                'date': latest_data.index[0].strftime('%Y-%m-%d'),
                'symbol': self.symbol,
                'current_price': current_price,
                'prediction': 'BUY' if prediction == 1 else 'SELL',
                'confidence': confidence,
                'confidence_level': 'High' if confidence >= 0.75 else ('Medium' if confidence >= 0.60 else 'Low'),
                'features': current_features.to_dict('records')[0]
            }

            return recommendation

        except Exception as e:
            logger.error(f"Failed to generate trading recommendation: {str(e)}")
            return {"error": str(e)}

    def calculate_performance_metrics(self, trades: List[Dict], equity_curve: List[Dict]) -> Dict[str, Any]:
        """Calculate accurate performance metrics."""
        try:
            # Korrekte Renditeberechnung
            strategy_returns = pd.Series([t['return'] for t in trades])
            strategy_equity = pd.Series([e['equity'] for e in equity_curve])

            # Buy and Hold Performance als Benchmark
            start_price = self.data['Close'].iloc[0]
            end_price = self.data['Close'].iloc[-1]
            benchmark_return = (end_price - start_price) / start_price
            benchmark_returns = self.data['Close'].pct_change().dropna()

            metrics = {
                'Strategy_Metrics': {
                    'total_return': float(strategy_equity.iloc[-1] / strategy_equity.iloc[0] - 1),
                    'annualized_return': self.calculate_annualized_return(strategy_returns),
                    'sharpe_ratio': self.calculate_sharpe_ratio(strategy_returns),
                    'sortino_ratio': self.calculate_sortino_ratio(strategy_returns),
                    'max_drawdown': self.calculate_max_drawdown(strategy_equity),
                    'profit_factor': self.calculate_profit_factor(trades),
                    'win_rate': len([t for t in trades if t['pnl'] > 0]) / len(trades) if trades else 0,
                    'avg_win': np.mean([t['pnl'] for t in trades if t['pnl'] > 0]) if trades else 0,
                    'avg_loss': abs(np.mean([t['pnl'] for t in trades if t['pnl'] < 0])) if trades else 0,
                    'trades_per_day': len(trades) / len(self.data) if len(self.data) > 0 else 0
                },

                'Benchmark_Metrics': {
                    'total_return': benchmark_return,
                    'annualized_return': self.calculate_annualized_return(benchmark_returns),
                    'sharpe_ratio': self.calculate_sharpe_ratio(benchmark_returns),
                    'max_drawdown': self.calculate_max_drawdown(self.data['Close'])
                }
            }

            return metrics

        except Exception as e:
            logger.error(f"Error calculating performance metrics: {str(e)}")
            return {}

    def calculate_trade_pnl(self, trade: Dict) -> float:
        """Calculate accurate P&L for a trade."""
        try:
            position_size = trade['size']
            entry_price = trade['entry_price']
            exit_price = trade['exit_price']
            commission = 0.001  # 0.1% pro Trade

            # Basis P&L
            if trade['type'] == 'BUY':
                gross_pnl = (exit_price - entry_price) * position_size
            else:  # SELL
                gross_pnl = (entry_price - exit_price) * position_size

            # Kommissionen berücksichtigen
            total_commission = (entry_price + exit_price) * position_size * commission
            net_pnl = gross_pnl - total_commission

            return net_pnl

        except Exception as e:
            logger.error(f"Error calculating trade P&L: {str(e)}")
            return 0.0

    def plot_renko_chart(self) -> None:
        """Plot Renko chart with signals and indicators."""
        try:
            fig, ax = plt.subplots(figsize=(15, 8))

            # Renko Bricks plotten
            for brick in self.renko.bricks:
                color = 'green' if brick['direction'] == 1 else 'red'
                ax.add_patch(plt.Rectangle(
                    (brick['timestamp'], brick['open']),
                    width=timedelta(days=1),
                    height=abs(brick['close'] - brick['open']),
                    facecolor=color,
                    alpha=0.5
                ))

            # Signale einzeichnen
            for signal in self.signals:
                marker = '^' if signal['type'] == 'BUY' else 'v'
                color = 'g' if signal['type'] == 'BUY' else 'r'
                ax.plot(
                    signal['timestamp'],
                    signal['price'],
                    marker=marker,
                    color=color,
                    markersize=10,
                    label=f"{signal['type']} Signal"
                )

            ax.set_title(f'Renko Chart - {self.symbol}')
            ax.set_xlabel('Date')
            ax.set_ylabel('Price')
            ax.grid(True)
            ax.legend()

            plt.savefig(f'{self.symbol}_renko_chart.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting Renko chart: {str(e)}")

    def set_dynamic_stop_loss(self, entry_price: float, signal_type: str,
                            volatility: float) -> float:
        """Set dynamic stop loss based on volatility and market regime."""
        try:
            base_stop = (entry_price - 2 * self.renko.brick_size
                        if signal_type == 'BUY'
                        else entry_price + 2 * self.renko.brick_size)

            vol_adjustment = volatility * np.sqrt(252) * entry_price

            regime = self.determine_market_regime(self.renko.bricks[-4:])
            regime_factor = {
                'STRONG_BULL': 1.5,
                'WEAK_BULL': 1.2,
                'CHOPPY_BULL': 1.0,
                'NEUTRAL': 0.8,
                'CHOPPY_BEAR': 1.0,
                'WEAK_BEAR': 1.2,
                'STRONG_BEAR': 1.5
            }.get(regime['trend_direction'], 1.0)

            if signal_type == 'BUY':
                return base_stop - (vol_adjustment * regime_factor)
            else:
                return base_stop + (vol_adjustment * regime_factor)

        except Exception as e:
            logger.error(f"Error setting stop loss: {str(e)}")
            return base_stop

    def manage_drawdown(self) -> bool:
        """Manage maximum drawdown."""
        try:
            if self.current_drawdown >= self.max_drawdown:
                logger.warning(f"Maximum drawdown ({self.max_drawdown:.1%}) reached. "
                             f"Closing all positions.")
                self.close_all_positions()
                return False
            return True
        except Exception as e:
            logger.error(f"Error in drawdown management: {str(e)}")
            return False

    def manage_portfolio_exposure(self, new_position_size: float,
                                price: float) -> float:
        """Manage portfolio exposure limits."""
        try:
            new_exposure = (self.current_exposure +
                          (new_position_size * price / self.account_size))

            if new_exposure > self.max_portfolio_exposure:
                allowed_increase = self.max_portfolio_exposure - self.current_exposure
                return allowed_increase * self.account_size / price

            return new_position_size

        except Exception as e:
            logger.error(f"Error in exposure management: {str(e)}")
            return 0.0

    def plot_performance_metrics(self) -> None:
        """Plot performance metrics visualization."""
        try:
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

            # Returns Distribution
            returns = pd.Series([t['return'] for t in self.positions])
            ax1.hist(returns, bins=50, alpha=0.5, color='blue', label='Returns')
            ax1.axvline(returns.mean(), color='red', linestyle='--', label='Mean')
            ax1.set_title('Returns Distribution')
            ax1.set_xlabel('Return')
            ax1.set_ylabel('Frequency')
            ax1.legend()

            # Cumulative Returns
            cum_returns = (1 + returns).cumprod()
            ax2.plot(cum_returns.index, cum_returns.values, color='blue')
            ax2.set_title('Cumulative Returns')
            ax2.set_xlabel('Trade')
            ax2.set_ylabel('Cumulative Return')
            ax2.grid(True)

            # Win/Loss Analysis
            wins = [t['pnl'] for t in self.positions if t['pnl'] > 0]
            losses = [t['pnl'] for t in self.positions if t['pnl'] < 0]
            ax3.boxplot([wins, losses], labels=['Wins', 'Losses'])
            ax3.set_title('Win/Loss Distribution')
            ax3.set_ylabel('P&L ($)')

            # Rolling Sharpe Ratio
            rolling_sharpe = self.calculate_rolling_sharpe(returns)
            ax4.plot(rolling_sharpe.index, rolling_sharpe.values, color='blue')
            ax4.set_title('Rolling Sharpe Ratio (60 days)')
            ax4.set_xlabel('Date')
            ax4.set_ylabel('Sharpe Ratio')
            ax4.grid(True)

            plt.tight_layout()
            plt.savefig(f'{self.symbol}_performance_metrics.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting performance metrics: {str(e)}")

    def plot_risk_metrics(self) -> None:
        """Plot risk metrics visualization."""
        try:
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

            # Rolling Volatility
            returns = pd.Series([t['return'] for t in self.positions])
            rolling_vol = returns.rolling(window=20).std() * np.sqrt(252)
            ax1.plot(rolling_vol.index, rolling_vol.values, color='red')
            ax1.set_title('Rolling Volatility (20 days)')
            ax1.set_ylabel('Annualized Volatility')

            # Value at Risk Analysis
            var_levels = [0.99, 0.95, 0.90]
            vars = [self.calculate_var(returns, level) for level in var_levels]
            ax2.bar(var_levels, vars)
            ax2.set_title('Value at Risk Analysis')
            ax2.set_xlabel('Confidence Level')
            ax2.set_ylabel('VaR')

            # Risk-Return Scatter
            risk = [t['risk_amount'] for t in self.positions]
            ret = [t['return'] for t in self.positions]
            ax3.scatter(risk, ret)
            ax3.set_title('Risk-Return Profile')
            ax3.set_xlabel('Risk Amount')
            ax3.set_ylabel('Return')

            # Drawdown Distribution
            drawdowns = self.calculate_all_drawdowns()
            ax4.hist(drawdowns, bins=30, color='purple', alpha=0.6)
            ax4.set_title('Drawdown Distribution')
            ax4.set_xlabel('Drawdown')
            ax4.set_ylabel('Frequency')

            plt.tight_layout()
            plt.savefig(f'{self.symbol}_risk_metrics.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting risk metrics: {str(e)}")

    def calculate_var(self, returns: pd.Series, confidence: float = 0.95) -> float:
        """Calculate Value at Risk."""
        try:
            return float(np.percentile(returns, (1 - confidence) * 100))
        except Exception as e:
            logger.error(f"Error calculating VaR: {str(e)}")
            return 0.0

    def calculate_all_drawdowns(self) -> List[float]:
        """Calculate all drawdowns for distribution analysis."""
        try:
            equity = pd.Series([p['equity'] for p in self.positions])
            rolling_max = equity.expanding().max()
            drawdowns = (equity - rolling_max) / rolling_max * 100
            return drawdowns.tolist()
        except Exception as e:
            logger.error(f"Error calculating drawdowns: {str(e)}")
            return []

    def calculate_benchmark_comparison(self, strategy_returns: pd.Series) -> Dict[str, float]:
        """Calculate comprehensive benchmark comparison metrics."""
        try:
            # Buy and Hold Performance
            benchmark_returns = self.data['Close'].pct_change().dropna()

            metrics = {
                'Strategy_vs_Benchmark': {
                    'outperformance': float(strategy_returns.cumsum().iloc[-1] -
                                          benchmark_returns.cumsum().iloc[-1]),
                    'information_ratio': self.calculate_information_ratio(
                        strategy_returns, benchmark_returns
                    ),
                    'beta': self.calculate_beta(strategy_returns, benchmark_returns),
                    'alpha': self.calculate_alpha(strategy_returns, benchmark_returns),
                    'correlation': strategy_returns.corr(benchmark_returns),
                    'tracking_error': np.std(strategy_returns - benchmark_returns) * np.sqrt(252)
                },

                'Risk_Adjusted_Metrics': {
                    'strategy_sharpe': self.calculate_sharpe_ratio(strategy_returns),
                    'benchmark_sharpe': self.calculate_sharpe_ratio(benchmark_returns),
                    'strategy_sortino': self.calculate_sortino_ratio(strategy_returns),
                    'benchmark_sortino': self.calculate_sortino_ratio(benchmark_returns),
                    'strategy_calmar': self.calculate_calmar_ratio(strategy_returns),
                    'benchmark_calmar': self.calculate_calmar_ratio(benchmark_returns)
                }
            }

            return metrics

        except Exception as e:
            logger.error(f"Error calculating benchmark comparison: {str(e)}")
            return {}

    def calculate_information_ratio(self, strategy_returns: pd.Series,
                                  benchmark_returns: pd.Series) -> float:
        """Calculate Information Ratio."""
        try:
            excess_returns = strategy_returns - benchmark_returns
            tracking_error = excess_returns.std() * np.sqrt(252)
            return excess_returns.mean() / tracking_error if tracking_error != 0 else 0.0
        except Exception as e:
            logger.error(f"Error calculating information ratio: {str(e)}")
            return 0.0

    def calculate_beta(self, strategy_returns: pd.Series,
                      benchmark_returns: pd.Series) -> float:
        """Calculate strategy beta."""
        try:
            covariance = strategy_returns.cov(benchmark_returns)
            variance = benchmark_returns.var()
            return covariance / variance if variance != 0 else 1.0
        except Exception as e:
            logger.error(f"Error calculating beta: {str(e)}")
            return 1.0

    def calculate_alpha(self, strategy_returns: pd.Series,
                       benchmark_returns: pd.Series) -> float:
        """Calculate Jensen's Alpha."""
        try:
            risk_free_rate = 0.02/252  # 2% annualisiert
            beta = self.calculate_beta(strategy_returns, benchmark_returns)
            strategy_excess = strategy_returns.mean() - risk_free_rate
            market_excess = benchmark_returns.mean() - risk_free_rate
            return strategy_excess - (beta * market_excess)
        except Exception as e:
            logger.error(f"Error calculating alpha: {str(e)}")
            return 0.0

    def calculate_sortino_ratio(self, returns: pd.Series) -> float:
        """Calculate Sortino Ratio."""
        try:
            excess_returns = returns - 0.02/252
            downside_returns = returns[returns < 0]
            downside_std = np.sqrt(np.mean(downside_returns**2))
            return np.sqrt(252) * excess_returns.mean() / downside_std if downside_std != 0 else 0.0
        except Exception as e:
            logger.error(f"Error calculating Sortino ratio: {str(e)}")
            return 0.0

    def calculate_calmar_ratio(self, returns: pd.Series) -> float:
        """Calculate Calmar Ratio."""
        try:
            max_dd = self.calculate_max_drawdown(returns.cumsum())
            annual_return = returns.mean() * 252
            return annual_return / abs(max_dd) if max_dd != 0 else float('inf')
        except Exception as e:
            logger.error(f"Error calculating Calmar ratio: {str(e)}")
            return 0.0

    def get_current_price(self) -> float:
        """Get strictly real-time current market price."""
        try:
            # Erste Validierungsebene: Aktuelle Minute
            current_minute_data = yf.download(
                self.symbol,
                start=datetime.now() - timedelta(minutes=5),  # Nur letzte 5 Minuten
                end=datetime.now(),
                interval='1m',
                progress=False
            )

            if current_minute_data.empty:
                raise ValueError(f"No real-time data available for {self.symbol}")

            current_price = current_minute_data['Close'].iloc[-1]

            # Zweite Validierungsebene: Zeitstempel-Prüfung
            last_update = current_minute_data.index[-1]
            time_difference = (datetime.now() - last_update).total_seconds()

            if time_difference > 300:  # Älter als 5 Minuten
                raise ValueError(f"Data too old for {self.symbol}. Difference: {time_difference} seconds")

            logger.info(f"Verified real-time price for {self.symbol}: ${current_price:.2f}")
            logger.info(f"Time since last update: {time_difference:.2f} seconds")

            return current_price

        except Exception as e:
            logger.error(f"Error getting real-time price: {str(e)}")
            raise

    def validate_price_data(self, symbol: str) -> float:
        """Strict price validation with multiple sources and checks."""
        try:
            # 1. Primäre Datenquelle (Yahoo Finance)
            ticker = yf.Ticker(symbol)
            live_data = ticker.history(period='1d', interval='1m')

            if live_data.empty:
                raise ValueError(f"No live data available for {symbol}")

            current_price = live_data['Close'].iloc[-1]
            timestamp = live_data.index[-1]

            # 2. Zeitstempel-Validierung
            time_diff = (datetime.now(timestamp.tzinfo) - timestamp).total_seconds()
            if time_diff > 60:  # Maximal 1 Minute Verzögerung
                raise ValueError(f"Data too old: {time_diff} seconds")

            # 3. Plausibilitätsprüfung mit Tages-OHLC
            day_data = ticker.history(period='1d')
            daily_high = day_data['High'].iloc[-1]
            daily_low = day_data['Low'].iloc[-1]

            if not (daily_low <= current_price <= daily_high):
                raise ValueError("Price outside daily range")

            # 4. Volumen-Validierung
            if live_data['Volume'].iloc[-1] == 0:
                raise ValueError("No trading volume")

            # 5. Cross-Validierung mit zusätzlicher Quelle
            alternative_price = self.get_alternative_price(symbol)
            if abs((current_price - alternative_price) / current_price) > 0.01:
                raise ValueError("Price discrepancy between sources")

            # 6. Logging und Reporting
            logger.info(f"""
            Price Validation Report for {symbol}:
            - Current Price: ${current_price:.2f}
            - Timestamp: {timestamp}
            - Data Age: {time_diff:.1f} seconds
            - Daily Range: ${daily_low:.2f} - ${daily_high:.2f}
            - Alternative Price: ${alternative_price:.2f}
            - Validation: PASSED
            """)

            return current_price

        except Exception as e:
            logger.error(f"Price validation failed for {symbol}: {str(e)}")
            raise RuntimeError(f"Cannot proceed without validated price data")

    def get_alternative_price(self, symbol: str) -> float:
        """Get price from alternative source for cross-validation."""
        try:
            # Implementierung einer alternativen Datenquelle
            # (z.B. Alpha Vantage, IEX, etc.)
            pass
        except Exception as e:
            logger.error(f"Alternative price fetch failed: {str(e)}")
            raise

    def get_current_recommendation(self) -> Dict[str, Any]:
        """Generate current trading recommendation with validated price data."""
        try:
            current_price = self.get_current_price()  # Verwendet jetzt validierte Preisdaten

            # Rest der Methode bleibt unverändert
            latest_data = self.add_technical_indicators(self.stock_data).tail(1)

            if latest_data.empty:
                raise ValueError("No current data available")

            current_features = latest_data[self.features]
            X_scaled = self.scaler.transform(current_features)

            prediction = self.model.predict(X_scaled)[0]
            probability = self.model.predict_proba(X_scaled)[0]
            confidence = float(max(probability))

            recommendation = {
                'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'symbol': self.symbol,
                'current_price': current_price,  # Verwendet validierte Preisdaten
                'prediction': 'BUY' if prediction == 1 else 'SELL',
                'confidence': confidence,
                'confidence_level': 'High' if confidence >= 0.75 else ('Medium' if confidence >= 0.60 else 'Low')
            }

            return recommendation

        except Exception as e:
            logger.error(f"Failed to generate trading recommendation: {str(e)}")
            return {"error": str(e)}

    def track_transaction(self, trade_type: str, price: float, timestamp: datetime, exit_price: float = None) -> None:
        """Track trading transactions with entry and exit prices."""
        try:
            if trade_type in ['BUY', 'SELL']:
                transaction = {
                    'type': trade_type,
                    'entry_price': price,
                    'entry_time': timestamp,
                    'exit_price': exit_price,
                    'exit_time': None,
                    'pnl': None,
                    'market_regime': self.current_regime
                }

                self.transactions['trades_by_date'][date].append(transaction)

                logger.info(
                    f"Tracked {trade_type} transaction - Entry: ${price:.2f}" +
                    (f", Exit: ${exit_price:.2f}" if exit_price else "")
                )
        except Exception as e:
            logger.error(f"Error tracking transaction: {str(e)}")

    def get_transaction_history(self) -> List[Dict]:
        """Get complete transaction history with entry, exit details and cumulative P&L."""
        try:
            history = []
            cumulative_pnl = 0.0

            for position in self.closed_positions:
                pnl = position['pnl']
                cumulative_pnl += pnl if pnl else 0

                history.append({
                    'date_entry': position['entry_time'].strftime('%Y-%m-%d'),
                    'time_entry': position['entry_time'].strftime('%H:%M'),
                    'date_exit': position['exit_time'].strftime('%Y-%m-%d'),
                    'time_exit': position['exit_time'].strftime('%H:%M'),
                    'type': position['type'],
                    'entry_price': position['entry_price'],
                    'exit_price': position['exit_price'],
                    'pnl': position['pnl'],
                    'cumulative_pnl': cumulative_pnl,  # Neu hinzugefügt
                    'market_regime': position['market_regime']
                })

            # Füge offene Position hinzu, falls vorhanden
            if self.open_position:
                current_price = self.get_current_price()
                unrealized_pnl = self.calculate_pnl(
                    self.open_position['type'],
                    self.open_position['entry_price'],
                    current_price
                )
                temp_cumulative_pnl = cumulative_pnl + (unrealized_pnl if unrealized_pnl else 0)

                history.append({
                    'date_entry': self.open_position['entry_time'].strftime('%Y-%m-%d'),
                    'time_entry': self.open_position['entry_time'].strftime('%H:%M'),
                    'date_exit': 'Open',
                    'time_exit': 'Open',
                    'type': self.open_position['type'],
                    'entry_price': self.open_position['entry_price'],
                    'exit_price': current_price,
                    'pnl': unrealized_pnl,
                    'cumulative_pnl': temp_cumulative_pnl,  # Inkl. unrealisierter Gewinn/Verlust
                    'market_regime': self.open_position['market_regime']
                })

            return history

        except Exception as e:
            logger.error(f"Error getting transaction history: {str(e)}")
            return []

    def calculate_benchmark_performance(self) -> Dict[str, float]:
        """Calculate Buy-and-Hold performance for the same instrument."""
        try:
            # Buy and Hold Performance des gleichen Instruments
            start_price = self.data['Close'].iloc[0]
            end_price = self.data['Close'].iloc[-1]
            holding_period_returns = self.data['Close'].pct_change().dropna()

            benchmark_metrics = {
                'total_return': (end_price - start_price) / start_price,
                'annualized_return': self.calculate_annualized_return(holding_period_returns),
                'sharpe_ratio': self.calculate_sharpe_ratio(holding_period_returns),
                'max_drawdown': self.calculate_max_drawdown(self.data['Close']),
                'volatility': np.std(holding_period_returns) * np.sqrt(252),
                'start_price': start_price,
                'end_price': end_price,
                'holding_period_days': len(self.data)
            }

            return benchmark_metrics

        except Exception as e:
            logger.error(f"Error calculating benchmark performance: {str(e)}")
            return {}

    def get_comparative_analysis(self) -> Dict[str, float]:
        """Compare strategy performance with instrument's Buy-and-Hold strategy."""
        try:
            strategy_metrics = self.calculate_performance_metrics(
                self.positions,
                self.equity_curve
            )
            benchmark_metrics = self.calculate_benchmark_performance()

            comparative_analysis = {
                'strategy_return': strategy_metrics['total_return'],
                'benchmark_return': benchmark_metrics['total_return'],
                'outperformance': strategy_metrics['total_return'] - benchmark_metrics['total_return'],
                'strategy_sharpe': strategy_metrics['sharpe_ratio'],
                'benchmark_sharpe': benchmark_metrics['sharpe_ratio'],
                'relative_sharpe': strategy_metrics['sharpe_ratio'] - benchmark_metrics['sharpe_ratio'],
                'tracking_error': self.calculate_tracking_error(
                    pd.Series([t['return'] for t in self.positions]),
                    self.data['Close'].pct_change().dropna()
                ),
                'information_ratio': self.calculate_information_ratio(
                    pd.Series([t['return'] for t in self.positions]),
                    self.data['Close'].pct_change().dropna()
                )
            }

            return comparative_analysis

        except Exception as e:
            logger.error(f"Error in comparative analysis: {str(e)}")
            return {}

    def get_real_time_price(self, symbol: str) -> float:
        """Get strictly validated real-time price with multiple sources."""
        try:
            # Primäre Quelle
            ticker = yf.Ticker(symbol)
            live_data = ticker.history(period='1d', interval='1m')

            if live_data.empty:
                raise ValueError(f"No live data available for {symbol}")

            current_price = live_data['Close'].iloc[-1]

            # Zusätzliche Validierung mit Tages-OHLC
            day_data = ticker.history(period='1d')
            if abs((current_price - day_data['Close'].iloc[-1]) / day_data['Close'].iloc[-1]) > 0.01:
                logger.warning(f"Price validation failed for {symbol}")
                raise ValueError("Price validation failed")

            logger.info(f"Validated real-time price for {symbol}: ${current_price:.2f}")
            return current_price

        except ValueError as ve:
            logger.error(f"Validation error for {symbol}: {str(ve)}")
            raise
        except Exception as e:
            logger.error(f"Failed to get real-time price for {symbol}: {str(e)}")
            raise RuntimeError(f"Cannot proceed without valid real-time data")

    def generate_action_recommendation(self) -> Dict[str, Any]:
        """Generate colored action recommendation with active position check."""
        try:
            current_data = self.get_real_time_price(self.symbol)

            # Prüfe auf aktive Position
            active_position = self.get_active_position()
            if active_position:
                recommendation = {
                    'date': datetime.now().strftime('%Y-%m-%d'),
                    'time': datetime.now().strftime('%H:%M:%S'),
                    'action': 'HOLD',  # Wenn aktive Position, dann HOLD
                    'active_position': {
                        'entry_date': active_position['entry_time'].strftime('%Y-%m-%d'),
                        'entry_price': active_position['entry_price'],
                        'type': active_position['type']
                    },
                    'current_price': current_data
                }
            else:
                # Nur wenn keine aktive Position, neue Empfehlung generieren
                current_signal = self.analyze_current_pattern(self.renko.bricks[-4:])
                recommendation = {
                    'date': datetime.now().strftime('%Y-%m-%d'),
                    'time': datetime.now().strftime('%H:%M:%S'),
                    'action': current_signal,
                    'price': current_data,
                    'confidence': self.calculate_signal_confidence(self.renko.bricks[-4:], current_signal)
                }

            return recommendation

        except Exception as e:
            logger.error(f"Error generating recommendation: {str(e)}")
            return {}

    def get_previous_signals(self) -> List[Dict]:
        """Get history of previous trading signals."""
        try:
            signals = []
            for position in self.positions[-5:]:  # Letzte 5 Signale
                if position['type'] in ['BUY', 'SELL']:
                    signals.append({
                        'date': position['entry_time'].strftime('%Y-%m-%d'),
                        'action': position['type'],
                        'price': position['entry_price']
                    })
            return signals
        except Exception as e:
            logger.error(f"Error getting previous signals: {str(e)}")
            return []

    def print_analysis_summary(self):
        """Erweiterte Ausgabe mit Sentiment."""
        try:
            # Bisherige Analyse...

            # Sentiment-Analyse
            print("\nSENTIMENT ANALYSIS")
            print("=" * 80)
            print("LEGAL NOTICE:")
            print("All sentiment data sourced from Yahoo Finance public RSS feed.")
            print(f"Terms of Service: {self.sentiment_analyzer.source['terms_url']}")
            print("=" * 80 + "\n")

            sentiment = self.sentiment_analyzer.analyze_sentiment(self.symbol)

            print(f"Overall Sentiment Score: {sentiment['sentiment_score']:.2f}")
            print(f"Analysis Confidence: {sentiment['confidence']:.2%}")
            print(f"Sources Analyzed: {sentiment['source_count']}")
            print(f"Last Update: {sentiment['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}")

            # Rest der Analyse...

        except Exception as e:
            logger.error(f"Error printing analysis summary: {str(e)}")

    def generate_visualizations(self) -> None:
        """Generate and save all visualization plots."""
        try:
            # 1. Renko Chart mit Signalen
            self.plot_renko_chart()

            # 2. Performance Metriken
            self.plot_performance_metrics()

            # 3. Drawdown Analyse
            self.plot_drawdown_analysis()

            # 4. Regime Analyse
            self.plot_regime_analysis()

            # 5. Renditeverteilung
            self.plot_return_distribution()

            # 6. Risikometriken
            self.plot_risk_metrics()

            # 7. Effizienzanalyse
            self.plot_efficiency_analysis()

            # 8. Korrelationsmatrix
            self.plot_correlation_matrix()

            logger.info("All visualizations generated successfully")

        except Exception as e:
            logger.error(f"Error generating visualizations: {str(e)}")

    def plot_drawdown_analysis(self) -> None:
        """Plot drawdown analysis."""
        try:
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

            # Drawdown Verlauf
            equity_curve = pd.Series([p['equity'] for p in self.positions])
            rolling_max = equity_curve.expanding().max()
            drawdowns = (equity_curve - rolling_max) / rolling_max * 100

            ax1.plot(drawdowns.index, drawdowns.values, color='red')
            ax1.fill_between(drawdowns.index, drawdowns.values, 0, color='red', alpha=0.3)
            ax1.set_title('Drawdown History')
            ax1.set_xlabel('Trade')
            ax1.set_ylabel('Drawdown (%)')
            ax1.grid(True)

            # Drawdown Verteilung
            ax2.hist(drawdowns.values, bins=50, color='red', alpha=0.5)
            ax2.axvline(drawdowns.mean(), color='black', linestyle='--', label='Mean')
            ax2.set_title('Drawdown Distribution')
            ax2.set_xlabel('Drawdown (%)')
            ax2.set_ylabel('Frequency')
            ax2.legend()

            plt.tight_layout()
            plt.savefig(f'{self.symbol}_drawdown_analysis.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting drawdown analysis: {str(e)}")

    def plot_regime_analysis(self) -> None:
        """Plot market regime analysis."""
        try:
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

            # Regime Verteilung
            regime_counts = pd.Series([t['market_regime'] for t in self.positions]).value_counts()
            ax1.pie(regime_counts.values, labels=regime_counts.index, autopct='%1.1f%%')
            ax1.set_title('Market Regime Distribution')

            # Performance nach Regime
            regime_returns = pd.DataFrame({
                'regime': [t['market_regime'] for t in self.positions],
                'return': [t['return'] for t in self.positions]
            })
            regime_performance = regime_returns.groupby('regime')['return'].mean()
            ax2.bar(regime_performance.index, regime_performance.values)
            ax2.set_title('Average Return by Regime')
            ax2.set_ylabel('Average Return')
            plt.xticks(rotation=45)

            # Regime Übergänge
            regime_transitions = pd.Series([
                f"{self.positions[i]['market_regime']}->{self.positions[i+1]['market_regime']}"
                for i in range(len(self.positions)-1)
            ]).value_counts()
            ax3.bar(range(len(regime_transitions)), regime_transitions.values)
            ax3.set_title('Regime Transitions')
            ax3.set_ylabel('Frequency')
            plt.xticks(range(len(regime_transitions)), regime_transitions.index, rotation=45)

            # Regime Duration
            regime_durations = pd.Series([
                sum(1 for _ in group)
                for _, group in itertools.groupby([t['market_regime'] for t in self.positions])
            ])
            ax4.hist(regime_durations, bins=20)
            ax4.set_title('Regime Duration Distribution')
            ax4.set_xlabel('Duration (Trades)')
            ax4.set_ylabel('Frequency')

            plt.tight_layout()
            plt.savefig(f'{self.symbol}_regime_analysis.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting regime analysis: {str(e)}")

    def plot_return_distribution(self) -> None:
        """Plot return distribution analysis."""
        try:
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

            returns = pd.Series([t['return'] for t in self.positions])

            # Returns Histogram mit Normalverteilung
            from scipy import stats
            mu, sigma = stats.norm.fit(returns)
            x = np.linspace(returns.min(), returns.max(), 100)
            ax1.hist(returns, bins=50, density=True, alpha=0.5, color='blue')
            ax1.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', label='Normal Dist.')
            ax1.set_title('Return Distribution')
            ax1.legend()

            # Q-Q Plot
            stats.probplot(returns, dist="norm", plot=ax2)
            ax2.set_title('Q-Q Plot')

            # Autocorrelation
            pd.plotting.autocorrelation_plot(returns, ax=ax3)
            ax3.set_title('Return Autocorrelation')

            # Rolling Statistics
            rolling_mean = returns.rolling(window=20).mean()
            rolling_std = returns.rolling(window=20).std()
            ax4.plot(rolling_mean.index, rolling_mean, label='Mean')
            ax4.plot(rolling_std.index, rolling_std, label='Std')
            ax4.set_title('Rolling Statistics (20 periods)')
            ax4.legend()

            plt.tight_layout()
            plt.savefig(f'{self.symbol}_return_distribution.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting return distribution: {str(e)}")

    def plot_risk_metrics(self) -> None:
        """Plot risk metrics visualization."""
        try:
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

            # Rolling Volatility
            returns = pd.Series([t['return'] for t in self.positions])
            rolling_vol = returns.rolling(window=20).std() * np.sqrt(252)
            ax1.plot(rolling_vol.index, rolling_vol.values, color='red')
            ax1.set_title('Rolling Volatility (20 days)')
            ax1.set_ylabel('Annualized Volatility')
            ax1.grid(True)

            # Value at Risk
            var_levels = [0.99, 0.95, 0.90]
            vars = [self.calculate_var(returns, level) for level in var_levels]
            ax2.bar([f"VaR {int(level*100)}%" for level in var_levels], vars)
            ax2.set_title('Value at Risk Analysis')
            ax2.set_ylabel('VaR')

            # Risk-Return Scatter
            risk = [t['risk_amount'] for t in self.positions]
            ret = [t['return'] for t in self.positions]
            ax3.scatter(risk, ret)
            ax3.set_title('Risk-Return Profile')
            ax3.set_xlabel('Risk Amount')
            ax3.set_ylabel('Return')

            # Rolling Sharpe vs Sortino
            rolling_sharpe = self.calculate_rolling_sharpe(returns)
            rolling_sortino = self.calculate_rolling_sortino(returns)
            ax4.plot(rolling_sharpe.index, rolling_sharpe, label='Sharpe')
            ax4.plot(rolling_sortino.index, rolling_sortino, label='Sortino')
            ax4.set_title('Rolling Risk Ratios')
            ax4.legend()

            plt.tight_layout()
            plt.savefig(f'{self.symbol}_risk_metrics.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting risk metrics: {str(e)}")

    def plot_efficiency_analysis(self) -> None:
        """Plot trading efficiency analysis."""
        try:
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

            # Trade Duration vs Return
            durations = [t['duration'].days for t in self.positions]
            returns = [t['return'] for t in self.positions]
            ax1.scatter(durations, returns)
            ax1.set_title('Trade Duration vs Return')
            ax1.set_xlabel('Duration (Days)')
            ax1.set_ylabel('Return')

            # Position Size vs Return
            sizes = [t['size'] for t in self.positions]
            ax2.scatter(sizes, returns)
            ax2.set_title('Position Size vs Return')
            ax2.set_xlabel('Position Size ($)')
            ax2.set_ylabel('Return')

            # MAE/MFE Analysis
            mae = [t['max_adverse_excursion'] for t in self.positions]
            mfe = [t['max_favorable_excursion'] for t in self.positions]
            ax3.scatter(mae, mfe)
            ax3.set_title('MAE vs MFE')
            ax3.set_xlabel('Maximum Adverse Excursion')
            ax3.set_ylabel('Maximum Favorable Excursion')

            # Efficiency Ratio
            efficiency = pd.Series([t['efficiency_ratio'] for t in self.positions])
            ax4.hist(efficiency, bins=50)
            ax4.axvline(efficiency.mean(), color='red', linestyle='--', label='Mean')
            ax4.set_title('Trade Efficiency Distribution')
            ax4.set_xlabel('Efficiency Ratio')
            ax4.legend()

            plt.tight_layout()
            plt.savefig(f'{self.symbol}_efficiency_analysis.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting efficiency analysis: {str(e)}")

    def plot_correlation_matrix(self) -> None:
        """Plot correlation matrix of various metrics."""
        try:
            # Erstelle DataFrame mit verschiedenen Metriken
            data = pd.DataFrame({
                'return': [t['return'] for t in self.positions],
                'duration': [t['duration'].days for t in self.positions],
                'size': [t['size'] for t in self.positions],
                'volatility': [t['entry_volatility'] for t in self.positions],
                'mae': [t['max_adverse_excursion'] for t in self.positions],
                'mfe': [t['max_favorable_excursion'] for t in self.positions],
                'efficiency': [t['efficiency_ratio'] for t in self.positions]
            })

            # Berechne Korrelationsmatrix
            corr = data.corr()

            # Plotte Heatmap
            plt.figure(figsize=(10, 8))
            sns.heatmap(corr, annot=True, cmap='RdYlBu', center=0)
            plt.title('Correlation Matrix of Trading Metrics')

            plt.tight_layout()
            plt.savefig(f'{self.symbol}_correlation_matrix.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting correlation matrix: {str(e)}")

    def validate_historical_data(self, data: pd.DataFrame) -> bool:
        """Validate historical price data."""
        try:
            if data.empty:
                raise ValueError("No historical data available")

            # Prfe Datenkontinuität
            time_gaps = data.index.to_series().diff().dropna()
            if time_gaps.max().total_seconds() > 86400:  # Max 1 Tag Lücke
                raise ValueError("Data contains significant gaps")

            # Prüfe auf unrealistische Preissprünge
            price_changes = data['Close'].pct_change().abs()
            if price_changes.max() > 0.2:  # Max 20% Änderung
                raise ValueError("Data contains suspicious price movements")

            # Prüfe Handelsvolumen
            if (data['Volume'] == 0).any():
                raise ValueError("Data contains periods with zero volume")

            return True

        except Exception as e:
            logger.error(f"Historical data validation failed: {str(e)}")
            return False

    def monitor_and_adjust_positions(self) -> None:
        """Überwacht und passt aktive Positionen dynamisch an."""
        try:
            for position in self.active_positions:
                # 1. Trend-Überprüfung (NEU)
                current_trend = self.trend_analyzer.analyze_trend_strength(self.data)

                # 2. Stop-Loss Anpassung (NEU)
                if current_trend['strength'] > 0.7:
                    self.exit_manager.adjust_trailing_stop(
                        position,
                        current_trend['strength']
                    )

                # 3. Position Sizing Anpassung (NEU)
                if self.should_adjust_position_size(position, current_trend):
                    new_size = self.adaptive_position_sizer.recalculate_size(
                        position,
                        current_trend
                    )
                    self.adjust_position_size(position, new_size)

        except Exception as e:
            logger.error(f"Error monitoring positions: {str(e)}")

    def calculate_enhanced_metrics(self) -> Dict[str, float]:
        """Berechnet erweiterte Performance-Metriken."""
        try:
            metrics = {
                # Bestehende Metriken (BEIBEHALTEN)
                'total_return': self.calculate_total_return(),
                'sharpe_ratio': self.calculate_sharpe_ratio(),
                'max_drawdown': self.calculate_max_drawdown(),

                # Neue Metriken (NEU)
                'trend_alignment': self.calculate_trend_alignment(),
                'signal_quality': self.calculate_signal_quality(),
                'position_efficiency': self.calculate_position_efficiency(),
                'risk_adjusted_return': self.calculate_risk_adjusted_return(),
                'market_regime_accuracy': self.calculate_regime_accuracy()
            }

            # Strategie-Optimierung basierend auf Metriken (NEU)
            self.strategy_optimizer.analyze_and_optimize(metrics)

            return metrics

        except Exception as e:
            logger.error(f"Error calculating enhanced metrics: {str(e)}")
            return {}

    def validate_and_adjust_strategy(self) -> None:
        """Validiert und passt die Strategie kontinuierlich an."""
        try:
            # 1. Performance-Validierung
            metrics = self.calculate_enhanced_metrics()

            # 2. Brick-Größen-Optimierung (NEU)
            optimal_brick_size = self.dynamic_brick_sizer.calculate_optimal_brick_size(
                self.data,
                volatility_window=20
            )
            self.renko.update_brick_size(optimal_brick_size)

            # 3. Signal-Generierung optimieren (NEU)
            if metrics['signal_quality'] < self.target_metrics['signal_quality']:
                self.enhanced_signal_generator.adjust_parameters(metrics)

            # 4. Position Sizing anpassen (NEU)
            if metrics['risk_adjusted_return'] < self.target_metrics['risk_adjusted_return']:
                self.adaptive_position_sizer.adjust_parameters(metrics)

            # 5. Exit-Strategien optimieren (NEU)
            if metrics['position_efficiency'] < self.target_metrics['position_efficiency']:
                self.exit_manager.optimize_exit_parameters(metrics)

        except Exception as e:
            logger.error(f"Error validating strategy: {str(e)}")

    def get_strategy_status(self) -> Dict[str, Any]:
        """Liefert detaillierten Status der optimierten Strategie."""
        try:
            return {
                # Basis-Status (BEIBEHALTEN)
                'current_position': self.get_current_position(),
                'performance_metrics': self.calculate_enhanced_metrics(),

                # Erweiterte Status-Informationen (NEU)
                'optimization_status': {
                    'brick_size': self.renko.brick_size,
                    'signal_parameters': self.enhanced_signal_generator.get_parameters(),
                    'position_sizing': self.adaptive_position_sizer.get_parameters(),
                    'exit_management': self.exit_manager.get_parameters()
                },

                # Strategie-Gesundheit (NEU)
                'strategy_health': {
                    'signal_quality': self.calculate_signal_quality(),
                    'position_efficiency': self.calculate_position_efficiency(),
                    'risk_management': self.assess_risk_management(),
                    'optimization_status': self.strategy_optimizer.get_status()
                }
            }

        except Exception as e:
            logger.error(f"Error getting strategy status: {str(e)}")
            return {}

    def analyze_market_conditions(self) -> Dict[str, Any]:
        """Erweiterte Analyse der Marktbedingungen."""
        try:
            # 1. Multi-Timeframe-Analyse (NEU)
            mtf_analysis = self.multi_timeframe_analyzer.analyze_timeframes(self.symbol)

            # 2. Trend-Stärke und Qualität (NEU)
            trend_analysis = self.trend_analyzer.analyze_trend_strength(self.data)

            # 3. Volatilitäts-Analyse (NEU)
            volatility = self.calculate_volatility_profile(self.data)

            # 4. Marktregime-Bestimmung (NEU)
            market_regime = self.determine_market_regime(
                mtf_analysis,
                trend_analysis,
                volatility
            )

            return {
                'market_regime': market_regime,
                'trend_analysis': trend_analysis,
                'volatility_profile': volatility,
                'timeframe_analysis': mtf_analysis,
                'risk_level': self.calculate_risk_level(market_regime, volatility)
            }

        except Exception as e:
            logger.error(f"Error analyzing market conditions: {str(e)}")
            return {}

    def adaptive_risk_management(self) -> None:
        """Dynamisches Risikomanagement basierend auf Marktbedingungen."""
        try:
            # 1. Marktanalyse
            market_conditions = self.analyze_market_conditions()

            # 2. Risikoanpassungen (NEU)
            if market_conditions['risk_level'] > self.risk_threshold:
                self.reduce_position_exposure()
                self.tighten_stop_losses()

            # 3. Opportunity Scoring (NEU)
            opportunity_score = self.calculate_opportunity_score(market_conditions)

            # 4. Position Sizing Anpassung (NEU)
            self.adjust_position_sizing_rules(
                market_conditions,
                opportunity_score
            )

        except Exception as e:
            logger.error(f"Error in risk management: {str(e)}")

    def performance_optimization_loop(self) -> None:
        """Kontinuierliche Performance-Optimierung."""
        try:
            # 1. Performance-Analyse
            current_metrics = self.calculate_enhanced_metrics()

            # 2. Strategie-Optimierung (NEU)
            if self.needs_optimization(current_metrics):
                # Brick-Größen-Optimierung
                self.optimize_brick_size()

                # Signal-Parameter-Optimierung
                self.optimize_signal_parameters()

                # Position-Sizing-Optimierung
                self.optimize_position_sizing()

                # Exit-Regeln-Optimierung
                self.optimize_exit_rules()

            # 3. Dokumentation (NEU)
            self.log_optimization_results()

        except Exception as e:
            logger.error(f"Error in optimization loop: {str(e)}")

    def generate_enhanced_report(self) -> Dict[str, Any]:
        """Generiert erweiterten Performance- und Strategie-Bericht."""
        try:
            return {
                # Performance-Metriken
                'performance': self.calculate_enhanced_metrics(),

                # Strategie-Status
                'strategy_status': {
                    'current_optimization': self.strategy_optimizer.get_status(),
                    'signal_quality': self.calculate_signal_quality(),
                    'position_efficiency': self.calculate_position_efficiency()
                },

                # Markt-Analyse
                'market_analysis': self.analyze_market_conditions(),

                # Risiko-Metriken
                'risk_metrics': {
                    'current_risk_level': self.calculate_current_risk(),
                    'position_risk': self.calculate_position_risk(),
                    'market_risk': self.calculate_market_risk()
                },

                # Optimierungs-Historie
                'optimization_history': self.get_optimization_history(),

                # Empfehlungen
                'recommendations': self.generate_strategy_recommendations()
            }

        except Exception as e:
            logger.error(f"Error generating report: {str(e)}")
            return {}

    def generate_final_recommendation(self) -> Dict[str, Any]:
        """Generiert finale Handlungsempfehlung basierend auf allen Analysen."""
        try:
            # 1. Aktuelle Position prüfen
            current_position = self.get_current_position()

            # 2. Prüfe auf aktive Signale
            if current_position:
                last_signal_date = current_position['entry_date']
                days_since_signal = (datetime.now() - last_signal_date).days

                # Wenn bereits eine aktive Position existiert
                if current_position['type'] == 'BUY':
                    action = 'HOLD'
                    message = f"HOLD (Active BUY position from {last_signal_date.strftime('%Y-%m-%d')})"
                elif current_position['type'] == 'SELL':
                    action = 'HOLD'
                    message = f"HOLD (Active SELL position from {last_signal_date.strftime('%Y-%m-%d')})"

                return {
                    'action': action,
                    'message': message,
                    'entry_price': current_position['entry_price'],
                    'current_price': self.get_current_price(),
                    'days_held': days_since_signal,
                    'unrealized_gain': self.calculate_unrealized_gain(current_position)
                }

            # Wenn keine aktive Position existiert, normale Signalgenerierung
            else:
                # Rest der Signalgenerierung...
                pass  # Platzhalter für Code
        except Exception as e:
            logger.error(f"Error generating recommendation: {str(e)}")
            return {
                'action': 'HOLD',
                'message': 'Error in signal generation',
                'error': str(e)
            }

    def visualize_predictions(self, predictions: Dict[str, Any]) -> None:
        """Visualisiert die Preisprognosen."""
        try:
            plt.figure(figsize=(15, 8))

            # Historische Daten
            plt.plot(self.data.index[-30:], self.data['Close'].tail(30),
                    label='Historical', color='blue')

            # Prognosen
            current_price = predictions['current_price']
            current_date = self.data.index[-1]

            colors = {3: 'green', 5: 'orange', 10: 'red'}

            for period, pred in predictions['predictions'].items():
                future_date = current_date + pd.Timedelta(days=period)
                plt.plot([current_date, future_date],
                        [current_price, pred['price']],
                        f'--{colors[period]}',
                        label=f'{period}-Day Forecast')

                # Konfidenzintervalle
                confidence = pred['confidence']
                plt.fill_between([current_date, future_date],
                               [current_price, pred['price'] * (1 - confidence)],
                               [current_price, pred['price'] * (1 + confidence)],
                               color=colors[period], alpha=0.1)

            plt.title('Price Predictions with Confidence Intervals')
            plt.xlabel('Date')
            plt.ylabel('Price')
            plt.legend()
            plt.grid(True)

            plt.savefig(f'{self.symbol}_predictions.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error visualizing predictions: {str(e)}")
            raise

    def generate_prediction_report(self, predictions: Dict[str, Any]) -> str:
        """Erstellt detaillierten Prognosebericht."""
        try:
            report = "\nPRICE PREDICTION REPORT\n"
            report += "=" * 80 + "\n\n"

            # Aktuelle Marktdaten
            report += f"Current Price: ${predictions['current_price']:.2f}\n"
            report += f"Analysis Timestamp: {predictions['timestamp']}\n\n"

            # Prognosen
            report += "PRICE PREDICTIONS:\n"
            report += "-" * 40 + "\n"

            for days, pred in predictions['predictions'].items():
                report += f"\n{days}-Day Forecast:\n"
                report += f"  Predicted Price: ${pred['price']:.2f}\n"
                report += f"  Expected Return: {pred['return']*100:.2f}%\n"
                report += f"  Confidence: {pred['confidence']*100:.2f}%\n"

                # Metriken für diesen Zeitraum
                metrics = predictions['metrics'][days]
                report += f"  Model Metrics:\n"
                report += f"    R² Score: {metrics['test']['r2']:.3f}\n"
                report += f"    MAPE: {metrics['test']['mape']*100:.2f}%\n"
                report += f"    MAE: ${metrics['test']['mae']:.2f}\n"

            # Modellqualität
            report += "\nMODEL QUALITY METRICS:\n"
            report += "-" * 40 + "\n"
            avg_r2 = np.mean([m['test']['r2'] for m in predictions['metrics'].values()])
            avg_mape = np.mean([m['test']['mape'] for m in predictions['metrics'].values()])

            report += f"Average R² Score: {avg_r2:.3f}\n"
            report += f"Average MAPE: {avg_mape*100:.2f}%\n"

            return report

        except Exception as e:
            logger.error(f"Error generating prediction report: {str(e)}")
            raise

    def analyze_with_predictions(self, data: pd.DataFrame) -> Dict[str, Any]:
        """Führt Analyse mit Preisprognosen durch."""
        try:
            # Basis-Analyse
            analysis = self.analyze_market_conditions()

            # Preisprognosen
            predictions = self.prediction_engine.generate_predictions(data)

            # Visualisierung
            self.prediction_engine.visualize_predictions(data, predictions)

            # Report generieren
            prediction_report = self.prediction_engine.generate_prediction_report(predictions)

            return {
                'market_analysis': analysis,
                'predictions': predictions,
                'report': prediction_report
            }

        except Exception as e:
            logger.error(f"Error in analysis with predictions: {str(e)}")
            raise

    def generate_transaction_metrics(self) -> Dict[str, Any]:
        """Generiert detaillierte Transaktions- und Vergleichsmetriken."""
        try:
            # Transaktionsanalyse
            transactions = self.get_all_transactions()
            cumulative_pnl = 0.0
            metrics = {
                'trades': [],
                'cumulative': [],
                'benchmark': []
            }

            strategy_return = (cumulative_pnl / start_price) * 100

            metrics['comparison'] = {
                'strategy_return': strategy_return,
                'buy_hold_return': buy_hold_return * 100,
                'outperformance': strategy_return - (buy_hold_return * 100),
                'win_rate': len([t for t in metrics['trades'] if t['pnl'] > 0]) / len(transactions),
                'avg_win': np.mean([t['pnl'] for t in metrics['trades'] if t['pnl'] > 0]),
                'avg_loss': np.mean([t['pnl'] for t in metrics['trades'] if t['pnl'] < 0]),
                'largest_win': max([t['pnl'] for t in metrics['trades']]),
                'largest_loss': min([t['pnl'] for t in metrics['trades']]),
                'profit_factor': abs(sum([t['pnl'] for t in metrics['trades'] if t['pnl'] > 0]) /
                                   sum([t['pnl'] for t in metrics['trades'] if t['pnl'] < 0]))
            }  # Schließende Klammer hinzugefügt

            return metrics

        except Exception as e:
            logger.error(f"Error generating metrics: {str(e)}")
            return {}

    def analyze_bricks(self, bricks: List[Dict]) -> float:
        """Analysiert Renko-Bricks für Signalstärke."""
        try:
            momentum = sum(b['direction'] for b in bricks)
            # Größe der Bricks
            size_factor = sum(abs(b['close'] - b['open']) for b in bricks) / (len(bricks) * self.renko.brick_size)  # Klammer geschlossen
            # Zeitliche Komponente
            time_factor = 1.0
            if len(bricks) > 1:
                time_diff = bricks[-1]['timestamp'] - bricks[0]['timestamp']
                time_factor = min(1.0, time_diff.total_seconds() / (86400 * 5))
            return (momentum * size_factor * time_factor)
        except Exception as e:
            logger.error(f"Error analyzing bricks: {str(e)}")
            return 0.0

    def handle_analysis_error(self, error: Exception) -> None:
        """Behandelt Analysefehler."""
        try:
            print(f"Error during analysis: {str(error)}")
            logger.error(f"Error in error handler: {str(error)}")
        except Exception as e:
            print("Analysis cancelled.")

    # Neue Visualisierungs-Implementierung
    def create_performance_visualizations(self) -> None:
        """Erstellt alle Performance-Visualisierungen."""
        try:
            # 1. Performance Comparison Chart
            self.plot_performance_comparison()

            # 2. Monthly Returns Heatmap
            self.plot_monthly_returns_heatmap()

            # 3. Drawdown Analysis
            self.plot_drawdown_analysis()

            # 4. Rolling Returns
            self.plot_rolling_returns()

            # 5. Trade Distribution
            self.plot_trade_distribution()

            # 6. Risk Metrics
            self.plot_risk_metrics()

        except Exception as e:
            logger.error(f"Error creating visualizations: {str(e)}")

    def plot_performance_comparison(self) -> None:
        """Erstellt Performance-Vergleichschart."""
        try:
            fig, ax = plt.subplots(figsize=(15, 8))

            # Strategie-Performance
            strategy_returns = pd.Series(
                [t['cumulative_pnl'] for t in self.metrics['cumulative']],
                index=[t['date'] for t in self.metrics['cumulative']]
            )
            ax.plot(strategy_returns.index, strategy_returns.values,
                    label='Renko Strategy', color='blue', linewidth=2)
            plt.savefig('performance_comparison.png')
        except Exception as e:
            logger.error(f"Error plotting performance comparison: {str(e)}")
            raise

    def plot_monthly_returns_heatmap(self) -> None:
        """Erstellt Heatmap der monatlichen Returns."""
        try:
            # Monatliche Returns berechnen
            monthly_returns = self.calculate_monthly_returns()

            # Heatmap erstellen
            fig, ax = plt.subplots(figsize=(15, 8))
            sns.heatmap(monthly_returns.pivot_table(index=monthly_returns.index.year,
                                                  columns=monthly_returns.index.month,
                                                  values='return'),
                       cmap='RdYlGn', center=0, annot=True, fmt='.1f')

            ax.set_title('Monthly Returns Heatmap (%)')
            ax.set_xlabel('Month')
            ax.set_ylabel('Year')

            plt.savefig(f'{self.symbol}_monthly_returns.png',
                       dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting monthly returns heatmap: {str(e)}")

    def plot_drawdown_analysis(self) -> None:
        """Erstellt Drawdown-Analyse-Chart."""
        try:
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

            # Drawdown-Verlauf
            drawdowns = self.calculate_drawdown_series()
            ax1.plot(drawdowns.index, drawdowns.values, color='red', linewidth=2)
            ax1.fill_between(drawdowns.index, drawdowns.values, 0,
                            color='red', alpha=0.3)
            ax1.set_title('Drawdown Analysis')
            ax1.set_ylabel('Drawdown (%)')
            ax1.grid(True)

            # Drawdown-Verteilung
            ax2.hist(drawdowns.values, bins=50, color='red', alpha=0.5)
            ax2.set_title('Drawdown Distribution')
            ax2.set_xlabel('Drawdown (%)')
            ax2.set_ylabel('Frequency')

            plt.tight_layout()
            plt.savefig(f'{self.symbol}_drawdown.png', dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            logger.error(f"Error plotting drawdown analysis: {str(e)}")

    def get_latest_news(self) -> List[Dict]:
        """Holt die 5 aktuellsten News von Yahoo Finance."""
        try:
            url = f"https://feeds.finance.yahoo.com/rss/2.0/headline?s={self.symbol}&region=US&lang=en-US"
            feed = feedparser.parse(url)

            latest_news = []
            for entry in feed.entries[:5]:  # Nur die 5 neuesten
                latest_news.append({
                    'title': entry.title,
                    'date': datetime.strptime(entry.published, '%a, %d %b %Y %H:%M:%S %z'),
                    'summary': entry.summary,
                    'link': entry.link,
                    'source': 'Yahoo Finance'
                })

            return latest_news

        except Exception as e:
            logger.error(f"Error fetching news: {str(e)}")
            return []

    def integrate_predictions_in_output(self, predictions: Dict[str, Any]) -> str:
        """Integriert Prognosen in die Hauptausgabe."""
        try:
            output = "\nPRICE PREDICTIONS\n"
            output += "=" * 80 + "\n\n"

            for period in sorted(predictions['predictions'].keys()):
                pred = predictions['predictions'][period]
                output += f"{period}-Day Forecast:\n"
                output += f"    Predicted Price: ${pred['price']:.2f}\n"
                output += f"    Expected Return: {pred['return']*100:.2f}%\n"
                output += f"    Confidence: {pred['confidence']*100:.2f}%\n"
                output += f"    Model Metrics:\n"
                output += f"        R² Score: {pred['metrics']['test_r2']:.3f}\n"
                output += f"        MAPE: {pred['metrics']['test_mape']*100:.2f}%\n\n"

            return output

        except Exception as e:
            logger.error(f"Error integrating predictions: {str(e)}")
            return "Error generating predictions output"

    def format_prediction_output(self, predictions: Dict[str, Any]) -> str:
        """Formatiert die Prognoseausgabe."""
        try:
            output = "\nPRICE PREDICTIONS\n"
            output += "=" * 80 + "\n\n"

            for period in sorted(predictions['predictions'].keys()):
                pred = predictions['predictions'][period]
                output += f"{period}-Day Forecast:\n"
                output += f"    Predicted Price: ${pred['price']:.2f}\n"
                output += f"    Expected Return: {pred['return']*100:.2f}%\n"
                output += f"    Confidence: {pred['confidence']*100:.2f}%\n"
                output += f"    Model Metrics:\n"
                output += f"        R² Score: {pred['metrics']['test_r2']:.3f}\n"
                output += f"        MAPE: {pred['metrics']['test_mape']*100:.2f}%\n\n"

            return output

        except Exception as e:
            logger.error(f"Error formatting predictions: {str(e)}")
            return "Error generating prediction output"

    def perform_sensitivity_analysis(self, data: pd.DataFrame) -> Dict[str, Any]:
        """Führt eine detaillierte Sensitivitätsanalyse durch."""
        try:
            return {
                'price_changes': self.analyze_price_sensitivity(data),
                'volume_impact': self.analyze_volume_sensitivity(data),
                'volatility_effect': self.analyze_volatility_sensitivity(data),
                'trend_strength': self.analyze_trend_sensitivity(data)
            }
        except Exception as e:
            logger.error(f"Error in sensitivity analysis: {str(e)}")
            raise

    def analyze_price_sensitivity(self, data: pd.DataFrame) -> Dict[str, float]:
        """Analysiert Sensitivität gegenüber Preisänderungen."""
        try:
            base_price = data['Close'].iloc[-1]
            price_ranges = [-0.05, -0.03, -0.01, 0.01, 0.03, 0.05]  # ±1%, 3%, 5%
            sensitivity = {}

            for change in price_ranges:
                test_price = base_price * (1 + change)
                impact = self.calculate_strategy_impact(test_price)
                sensitivity[f"{change*100:+.1f}%"] = impact

            return sensitivity
        except Exception as e:
            logger.error(f"Error in price sensitivity: {str(e)}")
            raise

    def analyze_volume_sensitivity(self, data: pd.DataFrame) -> Dict[str, float]:
        """Analysiert Sensitivität gegenüber Volumenänderungen."""
        try:
            base_volume = data['Volume'].iloc[-1]
            volume_ranges = [0.5, 0.75, 1.0, 1.25, 1.5]  # 50% - 150%
            sensitivity = {}

            for factor in volume_ranges:
                test_volume = base_volume * factor
                impact = self.calculate_volume_impact(test_volume)
                sensitivity[f"{factor*100:.0f}%"] = impact

            return sensitivity
        except Exception as e:
            logger.error(f"Error in volume sensitivity: {str(e)}")
            raise

    def analyze_volatility_sensitivity(self, data: pd.DataFrame) -> Dict[str, float]:
        """Analysiert Sensitivität gegenüber Volatilitätsänderungen."""
        try:
            current_volatility = self.calculate_volatility(data)
            volatility_ranges = [0.5, 0.75, 1.0, 1.25, 1.5]  # 50% - 150%
            sensitivity = {}

            for factor in volatility_ranges:
                test_volatility = current_volatility * factor
                impact = self.calculate_volatility_impact(test_volatility)
                sensitivity[f"{factor*100:.0f}%"] = impact

            return sensitivity
        except Exception as e:
            logger.error(f"Error in volatility sensitivity: {str(e)}")
            raise

    def analyze_trend_sensitivity(self, data: pd.DataFrame) -> Dict[str, float]:
        """Analysiert Sensitivität gegenüber Trendstärke."""
        try:
            current_trend = self.calculate_trend_strength(data)
            trend_ranges = [-2, -1, 0, 1, 2]  # Stark bearish bis stark bullish
            sensitivity = {}

            for strength in trend_ranges:
                test_trend = current_trend + strength
                impact = self.calculate_trend_impact(test_trend)
                sensitivity[f"Level {strength:+d}"] = impact

            return sensitivity
        except Exception as e:
            logger.error(f"Error in trend sensitivity: {str(e)}")
            raise

    def format_sensitivity_output(self, sensitivity: Dict[str, Any]) -> str:
        """Formatiert die Sensitivitätsanalyse für die Ausgabe."""
        try:
            output = "\nSENSITIVITY ANALYSIS\n"
            output += "=" * 80 + "\n\n"

            # Preis-Sensitivität
            output += "Price Sensitivity:\n"
            output += "-" * 20 + "\n"
            for change, impact in sensitivity['price_changes'].items():
                output += f"Price Change {change}: Impact = {impact:+.2f}%\n"

            # Volumen-Sensitivität
            output += "\nVolume Sensitivity:\n"
            output += "-" * 20 + "\n"
            for level, impact in sensitivity['volume_impact'].items():
                output += f"Volume Level {level}: Impact = {impact:+.2f}%\n"

            # Volatilitäts-Sensitivität
            output += "\nVolatility Sensitivity:\n"
            output += "-" * 20 + "\n"
            for level, impact in sensitivity['volatility_effect'].items():
                output += f"Volatility Level {level}: Impact = {impact:+.2f}%\n"

            # Trend-Sensitivität
            output += "\nTrend Sensitivity:\n"
            output += "-" * 20 + "\n"
            for strength, impact in sensitivity['trend_strength'].items():
                output += f"Trend {strength}: Impact = {impact:+.2f}%\n"

            return output

        except Exception as e:
            logger.error(f"Error formatting sensitivity output: {str(e)}")
            return "Error generating sensitivity analysis output"

class LegalNewsValidator:
    """Ensures 100% legal compliance for news data collection."""

    def __init__(self):
        self.legal_source = {
            'name': 'Yahoo Finance RSS',
            'url': 'https://feeds.finance.yahoo.com/rss/2.0/headline',
            'type': 'Public RSS Feed',
            'terms_url': 'https://policies.yahoo.com/us/en/yahoo/terms/product-atos/apifeed/index.htm',
            'access_type': 'Public',
            'requires_api_key': False,
            'requires_subscription': False
        }

    def validate_news_source(self) -> bool:
        """Validate that news source is fully legal to use."""
        try:
            # Prüfe, ob Quelle öffentlich zugänglich ist
            if not self.legal_source['access_type'] == 'Public':
                return False

            # Prüfe, ob API-Key benötigt wird
            if self.legal_source['requires_api_key']:
                return False

            # Prüfe, ob Subscription benötigt wird
            if self.legal_source['requires_subscription']:
                return False

            return True

        except Exception as e:
            logger.error(f"Error validating news source: {str(e)}")
            return False

    def validate_news_links(self, news_items: List[Dict]) -> List[Dict]:
        """Validate news links and ensure public accessibility."""
        try:
            import requests
            validated_news = []

            for news in news_items:
                try:
                    # Prüfe Link-Verfügbarkeit
                    response = requests.head(news['link'], timeout=5)

                    # Prüfe auf öffentliche Zugänglichkeit
                    if response.status_code == 200 and 'subscription' not in response.url:
                        news['link_status'] = 'VALID (Public Access)'
                        news['access_type'] = 'Public'
                        validated_news.append(news)
                    else:
                        logger.warning(f"Skipping non-public link: {news['link']}")

                except Exception as e:
                    logger.error(f"Error validating link: {news['link']} - {str(e)}")

            return validated_news

        except Exception as e:
            logger.error(f"Error in link validation: {str(e)}")
            return []

    def generate_legal_notice(self) -> str:
        """Generate detailed legal notice for news usage."""
        return f"""
        LEGAL NOTICE FOR NEWS DATA USAGE
        ===============================
        Source: {self.legal_source['name']}
        Type: {self.legal_source['type']}
        Access: Public RSS Feed

        This data is sourced exclusively from Yahoo Finance's public RSS feed.
        - No API key required
        - No subscription required
        - Publicly accessible data only
        - Compliant with Yahoo's terms of service

        Terms of Service: {self.legal_source['terms_url']}

        All news items and links are validated for:
        1. Public accessibility
        2. No subscription requirements
        3. Legal compliance with terms of service

        Last Validation: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        """

class LatestNewsCollector:
    """Collects latest news from Yahoo Finance public RSS feed."""

    def __init__(self):
        self.source = {
            'name': 'Yahoo Finance RSS',
            'url': 'https://feeds.finance.yahoo.com/rss/2.0/headline',
            'legal_status': 'Public RSS Feed',
            'terms_url': 'https://policies.yahoo.com/us/en/yahoo/terms/product-atos/apifeed/index.htm'
        }

    def get_latest_news(self, symbol: str, limit: int = 5) -> List[Dict]:
        """Get latest news for symbol from Yahoo Finance."""
        try:
            url = f"{self.source['url']}?s={symbol}&region=US&lang=en-US"
            feed = feedparser.parse(url)

            news_items = []
            for entry in feed.entries[:limit]:
                news_items.append({
                    'title': entry.title,
                    'date': datetime.strptime(entry.published, '%a, %d %b %Y %H:%M:%S %z'),
                    'summary': entry.summary,
                    'link': entry.link,
                    'source': 'Yahoo Finance',
                    'source_url': url
                })

            return sorted(news_items, key=lambda x: x['date'], reverse=True)

        except Exception as e:
            logger.error(f"Error fetching news: {str(e)}")
            return []

    def format_news_output(self, news_items: List[Dict]) -> str:
        """Format news items for display."""
        output = "\nLATEST NEWS\n"
        output += "=" * 80 + "\n"

        # Legal Notice
        output += "LEGAL NOTICE:\n"
        output += "All news items are sourced from Yahoo Finance's public RSS feed.\n"
        output += "This is a freely available, public data source.\n"
        output += f"Terms of Service: {self.source['terms_url']}\n"
        output += "=" * 80 + "\n\n"

        for i, item in enumerate(news_items, 1):
            output += f"{i}. {item['title']}\n"
            output += f"   Date: {item['date'].strftime('%Y-%m-%d %H:%M:%S')}\n"
            output += f"   Summary: {item['summary']}\n"
            output += f"   Source: Yahoo Finance\n"
            output += f"   Link: {item['link']}\n"
            output += "-" * 80 + "\n"

        return output

    def validate_news_links(self, news_items: List[Dict]) -> List[Dict]:
        """Validate that all news links are working."""
        try:
            validated_news = []

            for news in news_items:
                try:
                    # Überprüfe Link-Verfügbarkeit
                    response = requests.head(news['link'], timeout=5)
                    if response.status_code == 200:
                        news['link_status'] = 'VALID'
                    else:
                        news['link_status'] = f'INVALID (Status: {response.status_code})'
                        logger.warning(f"Invalid link found: {news['link']}")
                except Exception as e:
                    news['link_status'] = f'ERROR ({str(e)})'
                    logger.error(f"Error validating link: {news['link']}")

                validated_news.append(news)

            return validated_news

        except Exception as e:
            logger.error(f"Error in link validation: {str(e)}")
            return news_items

class PricePredictionEngine:
    """Hochperformantes Prognosemodell für Kurswertvorhersagen."""

    def __init__(self):
        self.prediction_periods = [3, 5, 10]  # Werktage
        self.models = {}
        self.metrics = {}
        self.features = [
            'close', 'volume', 'rsi', 'macd', 'bollinger',
            'atr', 'adx', 'obv', 'mom', 'stoch'
        ]

    def prepare_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """Erstellt fortgeschrittene Features für das Modell."""
        try:
            df = data.copy()

            # Technische Indikatoren
            df['rsi'] = ta.momentum.RSIIndicator(df['Close']).rsi()
            df['macd'] = ta.trend.MACD(df['Close']).macd()
            df['bollinger'] = ta.volatility.BollingerBands(df['Close']).bollinger_mavg()
            df['atr'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close']).average_true_range()
            df['adx'] = ta.trend.ADXIndicator(df['High'], df['Low'], df['Close']).adx()
            df['obv'] = ta.volume.OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()
            df['mom'] = ta.momentum.ROCIndicator(df['Close']).roc()
            df['stoch'] = ta.momentum.StochasticOscillator(df['High'], df['Low'], df['Close']).stoch()

            # Lag Features
            for period in [1, 3, 5, 10]:
                df[f'return_{period}d'] = df['Close'].pct_change(period)
                df[f'volume_{period}d'] = df['Volume'].pct_change(period)

            # Volatilitäts Features
            df['volatility'] = df['Close'].rolling(window=20).std()
            df['volatility_ratio'] = df['volatility'] / df['volatility'].rolling(window=100).mean()

            return df.dropna()

        except Exception as e:
            logger.error(f"Error preparing features: {str(e)}")
            raise

    def build_prediction_model(self, prediction_days: int) -> Any:
        """Erstellt optimiertes Vorhersagemodell."""
        try:
            # Ensemble aus mehreren Modellen
            models = {
                'lgbm': LGBMRegressor(
                    n_estimators=1000,
                    learning_rate=0.01,
                    max_depth=15,
                    num_leaves=31,
                    random_state=42
                ),
                'xgb': XGBRegressor(
                    n_estimators=1000,
                    learning_rate=0.01,
                    max_depth=10,
                    random_state=42
                ),
                'catboost': CatBoostRegressor(
                    iterations=1000,
                    learning_rate=0.01,
                    depth=10,
                    random_state=42,
                    verbose=False
                )
            }

            return StackingRegressor(
                estimators=[(name, model) for name, model in models.items()],
                final_estimator=Ridge(alpha=1.0)
            )

        except Exception as e:
            logger.error(f"Error building prediction model: {str(e)}")
            raise

    def train_and_evaluate(self, data: pd.DataFrame, prediction_days: int) -> Dict[str, Any]:
        """Trainiert und evaluiert das Prognosemodell."""
        try:
            # Feature Preparation
            features_df = self.prepare_features(data)

            # Train-Test Split
            train_size = int(len(features_df) * 0.8)  # Überflüssige Klammer entfernt
            train_data = features_df.iloc[:train_size]
            test_data = features_df.iloc[train_size:-prediction_days]

            return {
                'model': model,
                'metrics': metrics,
                'feature_importance': self.get_feature_importance(model)
            }  # Geschweifte Klammer geschlossen

        except Exception as e:  # except-Block hinzugefügt
            logger.error(f"Error in train_and_evaluate: {str(e)}")
            raise

    def generate_predictions(self, data: pd.DataFrame) -> Dict[str, Any]:
        """Generiert Prognosen für alle Zeiträume."""
        try:
            predictions = {}
            current_price = data['Close'].iloc[-1]
            features = self.prepare_features(data)

            for period in self.prediction_periods:
                if period not in self.models:
                    self.train_model(data, period)

                model = self.models[period]
                current_features = features.iloc[-1:][self.features]
                pred_return = model.predict(current_features)[0]
                pred_price = current_price * (1 + pred_return)

                predictions[period] = {
                    'price': pred_price,
                    'return': pred_return,
                    'confidence': self.calculate_prediction_confidence(period),
                    'metrics': self.metrics[period]
                }

            return {
                'predictions': predictions,
                'current_price': current_price,
                'timestamp': datetime.now()
            }

        except Exception as e:
            logger.error(f"Error generating predictions: {str(e)}")
            raise

class SentimentAnalyzer:
    """Analysiert das Sentiment für ein Finanzinstrument."""

    def __init__(self):
        self.source = {
            'name': 'Yahoo Finance RSS',
            'url': 'https://feeds.finance.yahoo.com/rss/2.0/headline',
            'legal_status': 'Public RSS Feed',
            'terms_url': 'https://policies.yahoo.com/us/en/yahoo/terms/product-atos/apifeed/index.htm'
        }

    def analyze_sentiment(self, symbol: str) -> Dict[str, Any]:
        """Führt Sentiment-Analyse durch."""
        try:
            # News abrufen
            news_items = self.get_latest_news(symbol)

            # Sentiment berechnen
            sentiments = []
            for item in news_items:
                sentiment = self.calculate_sentiment(item['title'] + ' ' + item['summary'])
                sentiments.append(sentiment)

            # Gesamtsentiment
            if sentiments:
                avg_sentiment = sum(sentiments) / len(sentiments)
            else:
                avg_sentiment = 0.0

            return {
                'symbol': symbol,
                'sentiment_score': avg_sentiment,
                'confidence': min(len(sentiments) / 10, 1.0),
                'source_count': len(news_items),
                'timestamp': datetime.now()
            }

        except Exception as e:
            logger.error(f"Error in sentiment analysis: {str(e)}")
            return {
                'symbol': symbol,
                'sentiment_score': 0.0,
                'confidence': 0.0,
                'source_count': 0,
                'timestamp': datetime.now()
            }

if __name__ == "__main__":
    # GUI fr Symboleingabe starten
    gui = SymbolInputGUI()
    params = gui.get_parameters()

    if params:
        try:
            print(f"\nStarting Renko Trading Analysis for {params['symbol']}")
            print("=" * 80)

            # Strategy initialisieren
            strategy = RenkoStrategy(
                symbol=params['symbol'],
                atr_period=params['atr_period']
            )

            # Aktuelle Handelsempfehlung generieren
            recommendation = strategy.get_current_prediction()

            print("\nCurrent Trading Recommendation:")
            print("-" * 40)
            print(f"Symbol: {recommendation['symbol']}")
            print(f"Current Price: ${recommendation['current_price']:.2f}")
            print(f"Signal: {recommendation['signal']}")
            print(f"Confidence: {recommendation['confidence']:.2f}")

            if 'action' in recommendation:
                print(f"\nRecommended Action: {recommendation['action']}")

            # Performance Metriken berechnen
            metrics = strategy.calculate_performance_metrics(
                strategy.positions,
                strategy.equity_curve
            )

            print("\nStrategy Performance:")
            print("-" * 30)
            for metric, value in metrics.items():
                print(f"{metric.replace('_', ' ').title()}: {value:.2%}")

            # Visualisierungen generieren
            strategy.plot_renko_chart()

            print("\nVisualization files saved:")
            print(f"1. {params['symbol']}_renko.png")

        except Exception as e:
            print(f"Error during analysis: {str(e)}")
    else:
        print("Analysis cancelled.")

OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)